In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Load curves

In [2]:
def load_curves_sg(assess_qs, val_sizes, res_path):
    # read training curves
    curves = {}
    errors = {}
    for i in range(len(val_sizes)):
        curves_i = np.loadtxt(res_path+'curves/train_'+str(i)+'.txt')
        j = 0
        for quantity in assess_qs:
            if i==0:
                curves[quantity] = curves_i[j::8].reshape((1, -1))
                errors[quantity] = curves_i[j+1::8].reshape((1, -1))
            else:
                curves[quantity] = np.concatenate((curves[quantity], [curves_i[j::8]]), axis=0)
                errors[quantity] = np.concatenate((errors[quantity], [curves_i[j+1::8]]), axis=0)
            j += 2
            
    return curves, errors

## Similarity graph

In [4]:
# common params
assess_qs = ['min_acc', 'val_acc', 'missed', 'false_alarm']
val_sizes = [30]

# sg params
res_path = '../../data/experiments/slowfast/slowfast_sg/2023_04_11/'

sg_curves, sg_errors = load_curves_sg(assess_qs, val_sizes, res_path)

# print resutls
print('curves:\n', sg_curves)
print('\nerrors:\n', sg_errors)

curves:
 {'min_acc': array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]), 'val_acc': array([[0.67733333, 0.75333333, 0.698     , 0.78266667, 0.762     ,
        0.79933333]]), 'missed': array([[0.48      , 0.38933333, 0.41066667, 0.288     , 0.31333333,
        0.31333333]]), 'false_alarm': array([[0.16533333, 0.104     , 0.19333333, 0.14666667, 0.16266667,
        0.088     ]])}

errors:
 {'min_acc': array([[0., 0., 0., 0., 0., 0.]]), 'val_acc': array([[0.05591263, 0.02503331, 0.05663136, 0.03369141, 0.03762978,
        0.02398147]]), 'missed': array([[0.11933147, 0.03057959, 0.04706497, 0.04995998, 0.05448751,
        0.04501851]]), 'false_alarm': array([[0.06946142, 0.02370185, 0.09788883, 0.05129436, 0.03968767,
        0.0122202 ]])}


## Logistic regression

In [7]:
def load_curves_lgrg(assess_qs, val_sizes, res_path):
    curves = {}
    errors = {}  
    for i in range(len(val_sizes)):
        curves_i = np.loadtxt(res_path+'curves_'+str(i)+'.txt')
        curves_i = curves_i.reshape(8, -1)
        j = 0
        for quantity in assess_qs:
            if i == 0:
                curves[quantity] = curves_i[2*j].reshape(1, -1)
                errors[quantity] = curves_i[2*j+1].reshape(1, -1)
            else:
                curves[quantity] = np.concatenate((curves[quantity], [curves_i[2*j]]), axis=0)
                errors[quantity] = np.concatenate((errors[quantity], [curves_i[2*j+1]]), axis=0)
            j += 1
            
    return curves, errors

In [14]:
# get lgrg curves
res_path = '../../data/experiments/slowfast/slowfast_lgrg/2023_04_11-no regularization/'
lgrg_curves, lgrg_errors = load_curves_lgrg(assess_qs, val_sizes, res_path)

# print resutls
print('curves:\n', lgrg_curves)
print('\nerrors:\n', lgrg_errors)

curves:
 {'min_acc': array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]), 'val_acc': array([[0.754     , 0.74466667, 0.74266667, 0.768     , 0.758     ,
        0.76733333]]), 'missed': array([[0., 0., 0., 0., 0., 0.]]), 'false_alarm': array([[0.22      , 0.23866667, 0.22933333, 0.23733333, 0.23066667,
        0.25066667]])}

errors:
 {'min_acc': array([[0., 0., 0., 0., 0., 0.]]), 'val_acc': array([[0.03172101, 0.03000741, 0.01123487, 0.03297137, 0.03403266,
        0.01372751]]), 'missed': array([[0., 0., 0., 0., 0., 0.]]), 'false_alarm': array([[0.07682592, 0.04548993, 0.05941941, 0.03542755, 0.05376492,
        0.03946306]])}


# Plot curves

In [15]:
# group all curves
curves = [sg_curves, lgrg_curves]
errors = [sg_errors, lgrg_errors]

In [16]:
# plot curves
res_path = '../../data/experiments/comparison/2023_04_11/'
full_title = {'min_acc': 'minimum accuracy', 'val_acc': 'validation accuracy', 'missed': 'missed detection rate', 'false_alarm': 'false alarm rate'}
train_sizes = [50, 100, 150, 200, 250, 300]
val_sizes = [30]

if not os.path.exists(res_path):
    os.mkdir(res_path)

i = 0
for quantity in assess_qs:
    plt.figure(figsize=(8,4))
    
    # plot for all methods
    for m in range(len(curves)):        
        plt.errorbar(train_sizes, curves[m][quantity][i], errors[m][quantity][i])
    if quantity=='val_acc':
        plt.errorbar(train_sizes, sg_curves['min_acc'][i], sg_errors['min_acc'][i])
    
    # legend
    if quantity=='val_acc':
        plt.legend(['our method', 'logreg', 'min acc'])
    else:
        plt.legend(['our method', 'logreg'])
                    
    plt.xlabel('training set size')
    plt.ylabel(full_title[quantity])
    plt.ylim([0,1])
    _ = plt.title('validation set size = {}'.format(val_sizes[i]))
    plt.savefig(res_path+'curves_'+str(quantity)+'.png')
    plt.close()